In [1]:
!ls -l saved_model

total 0


In [2]:
import sys
sys.path.insert(0, "/workspace/recommendation")

import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

import torch.jit
import time
from argparse import ArgumentParser
import numpy as np
import torch

from neumf import NeuMF

In [3]:
def parse_args():
    parser = ArgumentParser(description="Benchmark inference performance of the NCF model")
    parser.add_argument('-f', '--file', help='Path for input file. First line should contain number of lines to search in')
    parser.add_argument('--load_checkpoint_path', default=None, type=str,
                        help='Path to the checkpoint file to be loaded before training/evaluation')
    parser.add_argument('--n_users', default=138493, type=int,
                        help='Number of users. Defaults to the number of users in the ml-20m dataset after preprocessing')
    parser.add_argument('--n_items', default=26744, type=int,
                        help='Number of items. Defaults to the number of users in the ml-20m dataset after preprocessing')
    parser.add_argument('-fac', '--factors', type=int, default=64,
                        help='Number of predictive factors')
    parser.add_argument('--dropout', type=float, default=0.5,
                        help='Dropout probability, if equal to 0 will not use dropout at all')
    parser.add_argument('--layers', nargs='+', type=int,
                        default=[256, 256, 128, 64],
                        help='Sizes of hidden layers for MLP')
    parser.add_argument('--batch_sizes', default='1,4,16,64,256,1024,4096,16384,65536,262144,1048576', type=str,
                        help='A list of comma-separated batch size values to benchmark')
    parser.add_argument('--num_batches', default=200, type=int,
                        help='Number of batches for which to measure latency and throughput')
    parser.add_argument('--fp16', action='store_true', help='Cast the model to FP16 precision', default=False)
    parser.add_argument('--log_path', default='log.json', type=str,
                        help='Path for the JSON training log')

    return parser.parse_args()

In [4]:
from scipy.spatial.distance import cdist
def find_similar_movies(nn_movie_id, item_embedding, item_embedding_norm=None, k=10):
    #if not item_embedding_norm:
    #    item_embedding_norm = np.linalg.norm(item_embedding, axis=1)
    #sim = np.dot(item_embedding, item_embedding[nn_movie_id].reshape(64, 1)).squeeze()/item_embedding_norm
    
    sim = 1-cdist(item_embedding, item_embedding[nn_movie_id].reshape(1, -1), metric="cosine")

    #sim = -cdist(item_embedding, item_embedding[nn_movie_id].reshape(1, -1), metric="euclidean")
    
    return sim.squeeze().argsort()[-k:][::-1]

In [5]:
args = parse_args()

In [19]:
model = NeuMF(nb_users=args.n_users, nb_items=args.n_items, mf_dim=args.factors,
              mlp_layer_sizes=args.layers, dropout=args.dropout)

model = model.cuda()

state_dict = torch.load("../saved_model/model.pth")
model.load_state_dict(state_dict)

model.eval()

NeuMF(
  (mf_user_embed): Embedding(138493, 64)
  (mf_item_embed): Embedding(26744, 64)
  (mlp_user_embed): Embedding(138493, 128)
  (mlp_item_embed): Embedding(26744, 128)
  (mlp): ModuleList(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
  )
  (final): Linear(in_features=128, out_features=1, bias=True)
)

In [20]:
model(torch.LongTensor([0]).cuda(),torch.LongTensor([0]).cuda(), sigmoid=True)

tensor([[0.8850]], device='cuda:0', grad_fn=<SigmoidBackward>)

# Calculate the AUC

In [22]:
from argparse import ArgumentParser
import pandas as pd
from load import implicit_load
import torch
import tqdm

In [71]:
def roc_auc_score(y_true, y_score):
    """ROC AUC score in PyTorch
    Args:
        y_true (Tensor):
        y_score (Tensor):
    """
    device = 'cpu'
    y_true.squeeze_()
    y_score.squeeze_()
    if y_true.shape != y_score.shape:
        raise TypeError(F"Shape of y_true and y_score must match. Got {y_true.shape()} and {y_score.shape()}.")

    desc_score_indices = torch.argsort(y_score, descending=True)
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]

    distinct_value_indices = torch.nonzero(y_score[1:] - y_score[:-1]).squeeze()
    threshold_idxs = torch.cat([distinct_value_indices, torch.tensor([y_true.numel() - 1], device=device)])

    tps = torch.cumsum(y_true, dim=0)[threshold_idxs]
    fps = 1 + threshold_idxs - tps

    tps = torch.cat([torch.zeros(1, device=device), tps])
    fps = torch.cat([torch.zeros(1, device=device), fps])

    fpr = fps / fps[-1]
    tpr = tps / tps[-1]

    area = torch.trapz(tpr, fpr).item()

    return area

In [24]:
MIN_RATINGS = 20
USER_COLUMN = 'user_id'
ITEM_COLUMN = 'item_id'

df = implicit_load('/data/ml-20m/ratings.csv', sort=False)

print("Filtering out users with less than {} ratings".format(MIN_RATINGS))
grouped = df.groupby(USER_COLUMN)
df = grouped.filter(lambda x: len(x) >= MIN_RATINGS)

print("Mapping original user and item IDs to new sequential IDs")
df[USER_COLUMN], unique_users = pd.factorize(df[USER_COLUMN])
df[ITEM_COLUMN], unique_items = pd.factorize(df[ITEM_COLUMN])

20000263 ratings on 26744 items from 138493 users from 1995-01-09 11:46:44 to 2015-03-31 06:40:02
Filtering out users with less than 20 ratings
Mapping original user and item IDs to new sequential IDs


In [25]:
# Need to sort before popping to get last item
df.sort_values(by='timestamp', inplace=True)

# clean up data
del df['rating'], df['timestamp']
df = df.drop_duplicates() # assuming it keeps order

# now we have filtered and sorted by time data, we can split test data out
grouped_sorted = df.groupby(USER_COLUMN, group_keys=False)
test_data = grouped_sorted.tail(1).sort_values(by='user_id')
# need to pop for each group
train_data = grouped_sorted.apply(lambda x: x.iloc[:-1])


In [44]:
test_data = test_data.set_index('user_id')

In [26]:
class _TestNegSampler:
    def __init__(self, train_ratings, nb_neg):
        self.nb_neg = nb_neg
        self.nb_users = int(train_ratings[:, 0].max()) + 1
        self.nb_items = int(train_ratings[:, 1].max()) + 1

        # compute unique ids for quickly created hash set and fast lookup
        ids = (train_ratings[:, 0] * self.nb_items) + train_ratings[:, 1]
        self.set = set(ids)

    def generate(self, batch_size=128*1024):
        users = torch.arange(0, self.nb_users).reshape([1, -1]).repeat([self.nb_neg, 1]).transpose(0, 1).reshape(-1)

        items = [-1] * len(users)

        random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
        print('Generating validation negatives...')
        for idx, u in enumerate(tqdm.tqdm(users.tolist())):
            if not random_items:
                random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
            j = random_items.pop()
            while u * self.nb_items + j in self.set:
                if not random_items:
                    random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
                j = random_items.pop()

            items[idx] = j
        items = torch.LongTensor(items)
        return items

In [29]:
sampler = _TestNegSampler(train_data.values, 100)  # using 100 negative samples
test_negs = sampler.generate().cuda()
test_negs = test_negs.reshape(-1, 100)

  1%|          | 80707/13849300 [00:00<00:17, 807059.22it/s]

Generating validation negatives...


100%|██████████| 13849300/13849300 [00:14<00:00, 953175.41it/s]


In [30]:
test_negs.shape

torch.Size([138493, 100])

In [59]:
y_true = np.zeros((test_negs.shape[0], test_negs.shape[1]+1))
y_pred = np.zeros((test_negs.shape[0], test_negs.shape[1]+1))

for user in tqdm.tqdm(range(test_negs.shape[0])):
    user_arr = user * np.ones((101), dtype=np.int64)
    item_arr = np.zeros((101), dtype=np.int64)
    item_arr[0:-1] = np.array(test_negs[user].cpu(), dtype=np.int64)
    item_arr[-1] = test_data.ix[user]['item_id']
    y_true[user,-1] =1
    y_pred[user,:] = model(torch.from_numpy(user_arr).cuda(),torch.from_numpy(item_arr).cuda(), sigmoid=True).cpu().detach().squeeze()
    

  0%|          | 0/138493 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
100%|██████████| 138493/138493 [02:48<00:00, 822.93it/s]


In [64]:
y_true

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [65]:
y_true = np.reshape(y_true,(-1,1))
y_pred = np.reshape(y_pred,(-1,1))

In [67]:
y_pred

array([[2.06238155e-06],
       [8.36760616e-09],
       [2.59773802e-08],
       ...,
       [5.91767095e-02],
       [3.17900092e-04],
       [8.80426526e-01]])

In [72]:
roc_auc_score(torch.from_numpy(y_true), torch.from_numpy(y_pred))

0.9832111306358946

# Item to item simiarity

In [ ]:
#find nearest neighbor
item_embedding = model.mf_item_embed.weight
item_embedding = item_embedding.detach().cpu().numpy()

import pickle

with open('./mappings.pickle', 'rb') as handle:
    movies_mapping = pickle.load(handle)["items"]

nn_to_movies = movies_mapping
movies_to_nn = {}
for i in range(len(movies_mapping)):
    movies_to_nn[movies_mapping[i]] = i

import pandas as pd
movies = pd.read_csv("/data/ml-20m/movies.csv", index_col="movieId")

movie_ID = 2
print("Query: ", movies.ix[movie_ID]["title"], movies.ix[movie_ID]["genres"])

print("Similar movies: ")
similar_movies = find_similar_movies(movies_to_nn[movie_ID], item_embedding)

for i in similar_movies:
    print(movies.ix[nn_to_movies[i]]["title"], movies.ix[nn_to_movies[i]]["genres"])

In [8]:
movies_index = np.unique(movies.index)
for i in range(1000):
    movie_ID = movies_index[i]
    print("Query: ", movies.ix[movie_ID]["title"], movies.ix[movie_ID]["genres"])

    print("Similar movies: ")
    similar_movies = find_similar_movies(movies_to_nn[movie_ID], item_embedding)

    for i in similar_movies:
        print(nn_to_movies[i], movies.ix[nn_to_movies[i]]["title"], movies.ix[nn_to_movies[i]]["genres"])
    print('=================================\n')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


Query:  Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
Similar movies: 
1 Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
260 Star Wars: Episode IV - A New Hope (1977) Action|Adventure|Sci-Fi
588 Aladdin (1992) Adventure|Animation|Children|Comedy|Musical
356 Forrest Gump (1994) Comedy|Drama|Romance|War
364 Lion King, The (1994) Adventure|Animation|Children|Drama|Musical|IMAX
1210 Star Wars: Episode VI - Return of the Jedi (1983) Action|Adventure|Sci-Fi
480 Jurassic Park (1993) Action|Adventure|Sci-Fi|Thriller
1073 Willy Wonka & the Chocolate Factory (1971) Children|Comedy|Fantasy|Musical
1265 Groundhog Day (1993) Comedy|Fantasy|Romance
1270 Back to the Future (1985) Adventure|Comedy|Sci-Fi

Query:  Jumanji (1995) Adventure|Children|Fantasy
Similar movies: 
2 Jumanji (1995) Adventure|Children|Fantasy
364 Lion King, The (1994) Adventure|Animation|Children|Drama|Musical|IMAX
586 Home Alone (1990) Children|Comedy
367 Mask, The (1994) Action|Comedy|Crime|Fantasy
5

175 Kids (1995) Drama
1483 Crash (1996) Drama|Thriller
166 Doom Generation, The (1995) Comedy|Crime|Drama
1447 Gridlock'd (1997) Crime
386 S.F.W. (1994) Drama
63 Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996) Comedy|Crime
27022 Thursday (1998) Action|Crime|Thriller

Query:  Ace Ventura: When Nature Calls (1995) Comedy
Similar movies: 
19 Ace Ventura: When Nature Calls (1995) Comedy
344 Ace Ventura: Pet Detective (1994) Comedy
231 Dumb & Dumber (Dumb and Dumber) (1994) Adventure|Comedy
153 Batman Forever (1995) Action|Adventure|Comedy|Crime
520 Robin Hood: Men in Tights (1993) Comedy
586 Home Alone (1990) Children|Comedy
370 Naked Gun 33 1/3: The Final Insult (1994) Action|Comedy
367 Mask, The (1994) Action|Comedy|Crime|Fantasy
500 Mrs. Doubtfire (1993) Comedy|Drama
2 Jumanji (1995) Adventure|Children|Fantasy

Query:  Money Train (1995) Action|Comedy|Crime|Drama|Thriller
Similar movies: 
20 Money Train (1995) Action|Comedy|Crime|Drama|Thriller
23 Assassi

452 Widows' Peak (1994) Drama
2801 Oscar and Lucinda (a.k.a. Oscar & Lucinda) (1997) Drama|Romance
301 Picture Bride (Bijo photo) (1994) Drama|Romance
211 Browning Version, The (1994) Drama
131 Frankie Starlight (1995) Drama|Romance
753 Month by the Lake, A (1995) Comedy|Drama|Romance
2943 Indochine (1992) Drama|Romance

Query:  Dead Man Walking (1995) Crime|Drama
Similar movies: 
36 Dead Man Walking (1995) Crime|Drama
25 Leaving Las Vegas (1995) Drama|Romance
300 Quiz Show (1994) Drama
475 In the Name of the Father (1993) Drama
1183 English Patient, The (1996) Drama|Romance|War
1357 Shine (1996) Drama|Romance
508 Philadelphia (1993) Drama
1120 People vs. Larry Flynt, The (1996) Comedy|Drama
1094 Crying Game, The (1992) Drama|Romance|Thriller
509 Piano, The (1993) Drama|Romance

Query:  Across the Sea of Time (1995) Documentary|IMAX
Similar movies: 
37 Across the Sea of Time (1995) Documentary|IMAX
33 Wings of Courage (1995) Adventure|Romance|IMAX
404 Brother Minister: The Assassinatio

1392 Citizen Ruth (1996) Comedy|Drama
305 Ready to Wear (Pret-A-Porter) (1994) Comedy
369 Mrs. Parker and the Vicious Circle (1994) Drama
1542 Brassed Off (1996) Comedy|Drama|Romance

Query:  Lamerica (1994) Adventure|Drama
Similar movies: 
53 Lamerica (1994) Adventure|Drama
645 Nelly & Monsieur Arnaud (1995) Drama
263 Ladybird Ladybird (1994) Drama
1406 Judgement in Stone, A (Cérémonie, La) (1995) Drama
583 Dear Diary (Caro Diario) (1994) Comedy|Drama
1575 Gabbeh (1996) Drama
1533 Promise, The (La promesse) (1996) Drama
326 To Live (Huozhe) (1994) Drama
214 Before the Rain (Pred dozhdot) (1994) Drama|War
80 White Balloon, The (Badkonake sefid) (1995) Children|Drama

Query:  Big Green, The (1995) Children|Comedy
Similar movies: 
54 Big Green, The (1995) Children|Comedy
575 Little Rascals, The (1994) Children|Comedy
274 Man of the House (1995) Comedy
1602 Leave It to Beaver (1997) Comedy
87 Dunston Checks In (1996) Children|Comedy
1822 Meet the Deedles (1998) Children|Comedy
6557 Born t

...

1949 Man for All Seasons, A (1966) Drama
1936 Mrs. Miniver (1942) Drama|War

Query:  Top Hat (1935) Comedy|Musical|Romance
Similar movies: 
945 Top Hat (1935) Comedy|Musical|Romance
907 Gay Divorcee, The (1934) Comedy|Musical|Romance
1934 You Can't Take It with You (1938) Comedy|Romance
935 Band Wagon, The (1953) Comedy|Musical
3097 Shop Around the Corner, The (1940) Comedy|Drama|Romance
947 My Man Godfrey (1936) Comedy|Romance
3606 On the Town (1949) Comedy|Musical|Romance
7055 Swing Time (1936) Comedy|Musical|Romance
3096 My Man Godfrey (1957) Comedy
7086 Pygmalion (1938) Comedy|Drama

Query:  To Be or Not to Be (1942) Comedy|Drama|War
Similar movies: 
946 To Be or Not to Be (1942) Comedy|Drama|War
2208 Lady Vanishes, The (1938) Drama|Mystery|Thriller
970 Beat the Devil (1953) Adventure|Comedy|Crime|Drama|Romance
936 Ninotchka (1939) Comedy|Romance
2936 Sullivan's Travels (1941) Adventure|Comedy|Romance
6650 Kind Hearts and Coronets (1949) Comedy|Drama
942 Laura (1944) Crime|Film-Noi

962 They Made Me a Criminal (I Became a Criminal) (They Made Me a Fugitive) (1939) Crime|Drama
1933 Life of Emile Zola, The (1937) Drama
1930 Cavalcade (1933) Drama|Romance|War
958 Lady of Burlesque (1943) Comedy|Mystery
2904 Rain (1932) Drama
959 Of Human Bondage (1934) Drama
974 Algiers (1938) Drama|Romance
925 Golden Earrings (1947) Adventure|Romance|War
1070 Macao (1952) Adventure
1929 Grand Hotel (1932) Drama|Romance

Query:  Inspector General, The (1949) Musical
Similar movies: 
963 Inspector General, The (1949) Musical
7075 Court Jester, The (1956) Adventure|Comedy|Musical
961 Little Lord Fauntleroy (1936) Drama
3086 March of the Wooden Soldiers (a.k.a. Babes in Toyland) (1934) Children|Comedy|Fantasy
943 Ghost and Mrs. Muir, The (1947) Drama|Fantasy|Romance
1083 Great Race, The (1965) Comedy|Musical
3061 Holiday Inn (1942) Comedy|Musical
2079 Kidnapped (1960) Adventure|Children|Drama
8651 Man of La Mancha (1972) Adventure|Comedy|Musical
1932 Great Ziegfeld, The (1936) Drama|Mus

981 Dangerous Ground (1997) Drama
732 Original Gangstas (1996) Crime
706 Sunset Park (1996) Drama
66320 11th Hour, The (2007) Documentary
675 Hostile Intentions (1994) Action|Drama|Thriller
545 Harem (1985) Drama
44815 ATL (2006) Drama
834 Phat Beach (1996) Comedy
626 Thin Line Between Love and Hate, A (1996) Comedy
70495 Kill Buljo: The Movie (2007) Action|Comedy

Query:  Picnic (1955) Drama
Similar movies: 
982 Picnic (1955) Drama
1940 Gentleman's Agreement (1947) Drama
1936 Mrs. Miniver (1942) Drama|War
959 Of Human Bondage (1934) Drama
1935 How Green Was My Valley (1941) Drama|Musical|Romance
1946 Marty (1955) Drama|Romance
3447 Good Earth, The (1937) Drama
3475 Place in the Sun, A (1951) Drama|Romance
4046 Friendly Persuasion (1956) Drama
971 Cat on a Hot Tin Roof (1958) Drama

Query:  Madagascar Skin (1995) Romance
Similar movies: 
983 Madagascar Skin (1995) Romance
763 Last of the High Kings, The (a.k.a. Summer Fling) (1996) Comedy|Drama
672 Tarantella (1995) Drama
109 Headless 

521 Romeo Is Bleeding (1993) Crime|Thriller
259 Kiss of Death (1995) Crime|Drama|Thriller
322 Swimming with Sharks (1995) Comedy|Drama
42 Dead Presidents (1995) Action|Crime|Drama

Query:  Curdled (1996) Crime
Similar movies: 
1000 Curdled (1996) Crime
2850 Public Access (1993) Drama|Thriller
600 Love and a .45 (1994) Action|Comedy|Crime
283 New Jersey Drive (1995) Crime|Drama
1715 Office Killer (1997) Thriller
18 Four Rooms (1995) Comedy
166 Doom Generation, The (1995) Comedy|Crime|Drama
614 Loaded (1994) Drama|Thriller
1447 Gridlock'd (1997) Crime
1483 Crash (1996) Drama|Thriller

Query:  Associate, The (Associé, L') (1979) Comedy
Similar movies: 
1001 Associate, The (Associé, L') (1979) Comedy
75446 Little Ashes (2008) Drama
5019 Queen Bee (1955) Drama
977 Moonlight Murder (1936) Mystery
814 Boy Called Hate, A (1995) Drama
2958 Naturally Native (1998) Drama
4075 Monkey's Tale, A (Château des singes, Le) (1999) Animation|Children
796 Very Natural Thing, A (1974) Drama
33318 In the Ti

1017 Swiss Family Robinson (1960) Adventure|Children
1018 That Darn Cat! (1965) Children|Comedy|Mystery
1012 Old Yeller (1957) Children|Drama
2057 Incredible Journey, The (1963) Adventure|Children
1016 Shaggy Dog, The (1959) Children|Comedy
1010 Love Bug, The (1969) Children|Comedy
26502 Christmas Carol, A (1984) Drama|Fantasy
1007 Apple Dumpling Gang, The (1975) Children|Comedy|Western
2035 Blackbeard's Ghost (1968) Children|Comedy
2016 Apple Dumpling Gang Rides Again, The (1979) Children|Comedy|Western

Query:  That Darn Cat! (1965) Children|Comedy|Mystery
Similar movies: 
1018 That Darn Cat! (1965) Children|Comedy|Mystery
1016 Shaggy Dog, The (1959) Children|Comedy
2047 Gnome-Mobile, The (1967) Adventure|Children|Fantasy|Musical
2017 Babes in Toyland (1961) Children|Fantasy|Musical
2032 Barefoot Executive, The (1971) Children|Comedy
1014 Pollyanna (1960) Children|Comedy|Drama
1017 Swiss Family Robinson (1960) Adventure|Children
2098 Son of Flubber (1963) Children|Comedy
6259 My Frie

In [ ]:
user_embedding = model.mf_user_embed.weight

In [ ]:
user_embedding.shape

In [ ]:
#item_embedding = model.mf_item_embed.weight

item_embedding = model.mlp_item_embed.weight

In [ ]:
torch.dot(item_embedding[0], item_embedding[1])

In [ ]:
#numpy
item_embedding = item_embedding.detach().cpu().numpy()

In [ ]:
item_embedding.shape

In [ ]:
item_embedding_norm = np.linalg.norm(item_embedding, axis=1)

In [ ]:
len(item_embedding_norm)

In [ ]:
sim = np.dot(item_embedding, item_embedding[0].reshape(-1, 1)).squeeze()

In [ ]:
sim

In [ ]:
len(sim/item_embedding_norm)

In [ ]:
sim.squeeze().argsort()[-10:][::-1]

In [ ]:
from scipy.spatial.distance import cdist

cosine_sim = 1-cdist(item_embedding, item_embedding[0].reshape(1, -1), metric="cosine")


In [ ]:
cosine_sim

In [ ]:
cosine_sim.shape

In [ ]:
cosine_sim.squeeze().argsort()[-10:][::-1]

In [ ]:
euclidean_sim = -cdist(item_embedding, item_embedding[0].reshape(1, -1), metric="euclidean")

In [ ]:
euclidean_sim

In [ ]:
!ls /data/ml-20m

In [ ]:
import pandas as pd
movies = pd.read_csv("/data/ml-20m/movies.csv", index_col="movieId")

In [ ]:
movies

In [ ]:
import pickle

with open('./mappings.pickle', 'rb') as handle:
    movies_mapping = pickle.load(handle)["items"]

In [ ]:
movies_mapping

In [ ]:
nn_to_movies = movies_mapping

In [ ]:
movies_to_nn = {}
for i in range(len(movies_mapping)):
    movies_to_nn[movies_mapping[i]] = i

In [ ]:
movies_to_nn

In [ ]:
import numpy as np
max(movies_mapping)

In [ ]:
sorted(movies_mapping)

In [ ]:
movies.ix[2]

In [ ]:
def find_similar_movies(nn_movie_id, item_embedding, item_embedding_norm=None, k=10):
    #if not item_embedding_norm:
    #    item_embedding_norm = np.linalg.norm(item_embedding, axis=1)
    #sim = np.dot(item_embedding, item_embedding[nn_movie_id].reshape(64, 1)).squeeze()/item_embedding_norm
    
    sim = 1-cdist(item_embedding, item_embedding[nn_movie_id].reshape(1, -1), metric="cosine")

    #sim = -cdist(item_embedding, item_embedding[nn_movie_id].reshape(1, -1), metric="euclidean")
    
    return sim.squeeze().argsort()[-k:][::-1]

In [ ]:
find_similar_movies(1, item_embedding)

In [ ]:
movie_ID = 1
print("Query: ", movies.ix[movie_ID]["title"], movies.ix[movie_ID]["genres"])

print("Similar movies: ")
similar_movies = find_similar_movies(movies_to_nn[movie_ID], item_embedding)

for i in similar_movies:
    print(movies.ix[nn_to_movies[i]]["title"], movies.ix[nn_to_movies[i]]["genres"])